In [1]:
#import sys
#sys.path.append('C:\\Users\\lucat\\PythonProjects\\financial-mathematic\\src')

import numpy as np
import pandas as pd

In [3]:
path = 'C:/Users/lucat/PythonProjects/financial-mathematic/data/estimated-revenue-losses.csv'
data = pd.read_csv(path, sep=';')

In [9]:
data = data.drop([4, 13, 17, 18])
data = data.iloc[:, :2]

In [10]:
data

,Category,Tax Justice Network 2021*
0,Austria,"0,12"
1,Belgium,"0,53"
2,Bulgaria,"0,02"
3,Croatia,"0,03"
5,Czechia,"0,04"
6,Denmark,"0,37"
7,Estonia,"0,05"
8,Finland,"0,21"
9,France,"0,27"
10,Germany,"0,25"
